In [1]:
#!pip install sentence_transformers
#!pip install transformers

import torch
import warnings
from utils.train_transformer import train_model
from utils.data_processing import get_one_hot, get_splitted_data
import pandas as pd
import os
warnings.filterwarnings('ignore')


c:\Users\Rashi\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BATCH_SIZE = 128
EPOCHS = 50
STEP_SIZE = 0.00001
EPS = .00000001 
WEIGHT_DECAY = 0
MODEL = 'xlm-roberta-base'
DATA_LEN = 16848
DATA_DIR = 'data'
DATA_FILE_NAME = 'Augmented_Data_EPI.csv'
CHECKPOINT_DIR = 'checkpoints'
MODEL_TYPES = ['self', 'parents', 'teacher', 're', 'cause']

OUTPUTS = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
def start_processes(MODEL_TYPES, DATA_LEN, CHECKPOINT_DIR, device, EPI_DATA_DIR, EPOCHS, STEP_SIZE, BATCH_SIZE, WEIGHT_DECAY, EPS, load_epi_model = False):
    data_epi = pd.read_csv(EPI_DATA_DIR)
    data_epi = get_one_hot(0, data_epi['epi'])    
    for model_type in MODEL_TYPES:
        data = get_splitted_data(model_type, data_epi)
        weights = [1, (data == 0).sum()/(data == 1).sum() * 3]
        model_weights = torch.FloatTensor(weights) 
        train_model(OUTPUTS,MODEL ,model_type, DATA_LEN , CHECKPOINT_DIR, device, EPI_DATA_DIR ,EPOCHS, STEP_SIZE, BATCH_SIZE, WEIGHT_DECAY, EPS, model_weights, load_model = load_epi_model)
    

In [4]:
start_processes(MODEL_TYPES, DATA_LEN, CHECKPOINT_DIR, device, os.path.join(DATA_DIR, DATA_FILE_NAME), EPOCHS, STEP_SIZE, BATCH_SIZE, WEIGHT_DECAY, EPS, load_epi_model = False)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

tensor([ 1.0000, 28.0252])
0
0


RuntimeError: CUDA out of memory. Tried to allocate 84.00 MiB (GPU 0; 8.00 GiB total capacity; 7.13 GiB already allocated; 0 bytes free; 7.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

MODEL = 'xlm-roberta-base'
#model_soc = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels = 6).to(device)
#model_soc.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, 'model_deberta34.ckpt'), map_location=device))
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model_epi1 = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels = 2).to(device)
model_epi1.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, 'model_deberta_self7.ckpt'), map_location=device))

model_epi2 = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels = 2).to(device)
model_epi2.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, 'model_deberta_self7.ckpt'), map_location=device))





